# Intro to TensorFlow
MDCS:ML
***

## Overview:
* [About TensorFlow](#About-TensorFlow)
    * [Installation](#Installation)
* [Computational graph](#Computational-graph)
    * [TensorBoard](#TensorBoard)
    * [Tensors](#Tensors)
* [Execution](#Execution)
* [Variables](#Variables)
* [Placeholders](#Placeholders)
* [Saving and restoring models](#Saving-and-restoring-models)
* [Useful links](#Useful-links)
***

## About TensorFlow

TensorFlow is a framework developed by Google Brain, used for Machine Learning.  
Can be used on CPU and GPU.  
High level APIs built on top of TensorFlow for building Neural Network models (Keras).  
For the future workshops we'll use low level TensorFlow, for better understending of implementation.

It's composed of:
  * Library – for defining computational graphs
  * Runtime – for executing them
***

### Installation

Assuming you have followed [setup for Numpy basics and Logistic Regression](https://msft.spoppe.com/:w:/t/mdcs/ESyb9JX7CcVIswGs0SftSMoBinWaEDKpn9n0qODsO77_Zw?rtime=Q9sF5npg1kg).

Activate your conda enviroment, and run:  
```pip install tensorflow```

If your machine has GPU:  
```pip install tensorflow-gpu```
***

## Computational graph

Describing computations as directed graph, with:
  * Edges - Tensors (represent values that will flow through graph)
  * Nodes - Operations (describe calculations that consume and produce tensors)
  
Example:  
![computational graph](./resources/computational_graph.jpg)
  

We're defining **dependency** of operations instead of **order**.  
Parts of graph that are not dependent of eachother can be ran in parallel.

Example in TensorFlow:

In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
a = tf.constant(3.0)
b = tf.constant(4.0)
c = tf.add(a, b)
d = a * b
e = d - c

We are creating graph as ```tf.Graph```.
When building graph, operations are just defined, not executed, and tensors don't hold any values yet.

In [ ]:
print(a)
print(c)
print(e)

***
### TensorBoard

Visualization [tool](https://www.tensorflow.org/guide/summaries_and_tensorboard) for TensorFlow.  
Should be installed with TensorFlow, if not just run ```pip install tensorboard``` in your conda enviroment.  
You can use TensorBoard to visualize graph, plot metrics, and show additional data like images.  

We are going to save our graph:

In [ ]:
# Directory where we'll save logs
log_dir = './logs/'

# Saving the computation graph to a TensorBoard summary file
writer = tf.summary.FileWriter(os.path.join(log_dir, 'graph_example'))
writer.add_graph(tf.get_default_graph())
writer.flush()

In a new terminal (with your conda enviroment activated) run:  
```tensorboard --logdir logs```  
and open link from the terminal (usually localhost:6006).
***

### Tensors

```python
tf.Tensor
```
Describes a set of primitive values shaped into an array of any number of dimensions.
Tensor has:
  * Rank – number of dimensions
  * Shape - a tuple of integers specifying the array's length along each dimension
```python
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```
Shape can be infered when building graph, without having any actual values.

In [ ]:
# Keeping the graph we built before we reset it
old_graph = tf.get_default_graph()
# Clears the default graph stack and resets the global default graph
tf.reset_default_graph()

In [ ]:
x = tf.zeros((5, 10))
y = tf.ones((2, 10))
z = tf.concat([x, y], axis=0)
_ = tf.print(z)

In [ ]:
# Saving the computation graph to a TensorBoard summary file
writer = tf.summary.FileWriter(os.path.join(log_dir, 'tensor_shape_example'))
writer.add_graph(tf.get_default_graph())
writer.flush()

You might need no refresh TensorBoard in your browser to see new graph.
***

## Execution

Now that we have defined our ```tf.Graph``` we can execute it by using ```tf.Session```.

```tf.Session``` is a class that represents a connection with runtime.
It owns physical resources so it's important to close the session after we're done with the execution.

```python
session.run(tensor) # evaluates tensor, and returns it's value
session.run([t1, t2, ...]) # evaluates all given tensors, and returns their values
```
It executes minimal part of graph to evaluate given tensors (only nodes that the tensors depend of).

In [ ]:
with tf.Session(graph=old_graph) as sess:
    print(sess.run(e))
    out_c, out_d = sess.run([c, d])
    print("Evaluated c: {}".format(out_c))
    print("Evaluated d: {}".format(out_d))

In [ ]:
with tf.Session() as sess:
    print("Evaluated x: {}".format(sess.run(x)))

You can check out TensorFlow's guide [Graphs and Sessions](https://www.tensorflow.org/guide/graphs) for more details.
***

## Variables

```python
tf.Variable
```
Variable represents a tensor whose value can be changed by running ops on it.  
They enable learning by perserving state across executions of the graph (they exist outside the context of a single ```session.run``` call).  

All trainable parameters are variables.

Varable is defined by:
  * Name
  * Type
  * Shape
  * Initializer
  
The best way to create a variable is to call the ```tf.get_variable``` function that requires variables name, other parameters are optional.
It gets an existing variable with these parameters or creates a new one.

In [ ]:
with old_graph.as_default():
    v = tf.get_variable("my_var", dtype=tf.float32, shape=[], initializer=tf.random_normal_initializer)

Before you can use a variable, it must be initialized.  
You can use ```tf.global_variables_initializer()``` to initialize all trainable variables.

All variables will be created as trainable, if you want to create non-trainable variables check out TensorFlow's [Variable collections](https://www.tensorflow.org/guide/variables#variable_collections) guide.

In [ ]:
with tf.Session(graph=old_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v))

Variables can be read and assigned:

In [ ]:
with old_graph.as_default():
    v2 = tf.get_variable("assigned_var", dtype=tf.float32, shape=[1,3], initializer=tf.zeros_initializer)
    inc_v2 = v2.assign(v2 + 1)
    init = tf.global_variables_initializer()

In [ ]:
var_sess = tf.Session(graph=old_graph)
var_sess.run(init)
print(var_sess.run(v))
print(var_sess.run(v2))
var_sess.run(inc_v2)
print(var_sess.run(v2))
var_sess.run(inc_v2)
print(var_sess.run(v2))
print(var_sess.run([inc_v2, v2]))
print(var_sess.run([v2, inc_v2])) # minimal part of graph is executed and then all values are returned

Note that variable exists and perseves value withing one ```tf.Session```, so you want to run the same session for the training.  
We can see our variables in the graph now:

In [ ]:
with old_graph.as_default():
    f = tf.add(e, v)
# Saving the computation graph to a TensorBoard summary file
writer = tf.summary.FileWriter(os.path.join(log_dir, 'variables_example'))
writer.add_graph(old_graph)
writer.flush()

TensorFlow's guide: [Variables](https://www.tensorflow.org/guide/variables).
***

## Placeholders

```python
tf.placeholder
```
A placeholder for tensor that will be fed in the execution.  
Can be used to add data in execution, instead of loading all data as constant for example.

In ```session.run``` you need to feed all placeholders that evaluated tensors depend on.

In [ ]:
tf.reset_default_graph()
p1 = tf.placeholder(tf.float32, shape=[3,3])
p2 = tf.placeholder(tf.float32, shape=[3,3])
r1 = tf.add(p1, p2)
r2 = tf.add(p1, p1)

In [ ]:
ones = np.ones((3,3))
twos = ones * 2
with tf.Session() as sess:
    print(sess.run(r1, feed_dict={p1: ones, p2: twos}))
    print(sess.run(r2, feed_dict={p1: twos}))

And if we don't feed the placeholder:

In [ ]:
with tf.Session() as sess:
    print(sess.run(r2)) # breaks

You can pass data as numpy arrays to placeholders, and handle everything else from python.  
TensorFlow also provides built in functionalities for working with data (you can check out [tf.data](https://www.tensorflow.org/guide/datasets_for_estimators)).
***

## Saving and restoring models

As part of training you can update variables in your graph, but they only exist within one session.  
To save values of your variables, you can use ```tf.train.Saver()```:

In [ ]:
with old_graph.as_default():
    saver = tf.train.Saver()

# Directory where we'll save models
models_dir = './models/'
model1_path = os.path.join(models_dir, 'model1.ckpt')

_ = saver.save(var_sess, model1_path)

Now we can restore saved model to a new session:

In [ ]:
with tf.Session(graph=old_graph) as new_sess:
    new_sess.run(init)
    print(new_sess.run(v))
    print(new_sess.run(v2))
    saver.restore(new_sess, model1_path)
    print(new_sess.run(v))
    print(new_sess.run(v2))

Saver will restore values to variables of the same name and shape, which means that not all variables from the saved model need to be restored.  
If you have some variables in current graph that do not exist in saved model, you need to specify which variables are restored, and initialize those that are not being restored.

In [ ]:
new_graph = tf.Graph()
with new_graph.as_default():
    var1 = tf.get_variable("my_var", shape=[])
    var2 = tf.get_variable("new_var", shape=[], initializer=tf.zeros_initializer)
    mul_vars = var1 * var2
    
    new_saver = tf.train.Saver({"my_var" : var1})

In [ ]:
with tf.Session(graph=new_graph) as sess:
#     var2.initializer.run() # uncomment to initialize
    new_saver.restore(sess, model1_path)
    print(sess.run(var1))
    print(sess.run(var2)) # not restored and never initialized

TensorFlow's guide [Save and Restore](https://www.tensorflow.org/guide/saved_model).
***

## Useful links

Beside above mentioned guides, here are some more resources you will find useful:
* [tf.Optimizer](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer) - TensorFlow provides automatic differentiation and different optimizers - gradiants for all variables in your graph are automatically computed and variables are updated.  
* [Train your first neural network: basic classification](https://www.tensorflow.org/tutorials/keras/basic_classification) - TensorFlow's tutorial
* [Introduction](https://www.tensorflow.org/guide/low_level_intro) - similar overview with more detailes
    * example of optimizer use in [Training](https://www.tensorflow.org/guide/low_level_intro#training_2) section
* [TensorFlow Playground](https://playground.tensorflow.org) - play with different parameters of a Neural Network and visualize training
***

#### We hope you found this intro helpful!
#### Thanks!